In [1]:
import numpy as np
import pandas as pd
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 10)

# Average dN/dS

In [2]:
# Define a function for calculating dN/dS when both dN and dS is zero. 
def weird_division(df):
    if df[5]==0 and df[6]==0:
        return 0
    elif df[5]==0:
        return 0
    elif df[6]==0:
        return np.NaN
    return df[5] / df[6]

Import ortholog information, including dN and dS, which was downloaded from Ensembl 98. 

In [3]:
df_list = []
for file in sorted(glob.glob('../results/Ensembl98_human/human_protein_coding_genes.*.txt')):
    species_code_name = (file[29:-4])
    # print(species_code_name)
    df = pd.read_csv(file, sep='\t', header=None, na_values=('ortholog_one2many', 'ortholog_many2many') ,index_col=0)
    df = pd.DataFrame(df.dropna().drop_duplicates().apply(weird_division, axis=1),columns=[species_code_name+'_dNdS'])
    df_list.append(df.dropna().drop_duplicates())

In [4]:
# information of all human protein coding genes, which was downloaded from Ensembl98 
info_df = pd.read_csv('../data/info.human_protein_coding_genes.tsv',sep='\t',header=0,index_col=0)

In [5]:
info_df.drop_duplicates(subset='Gene name',inplace=True) # Drop the duplicated gene names

Now merge each of the mammalian species' dN/dS values against human onto the information of human protein-coding genes. 

In [7]:
integrate_df = info_df.copy(deep=True)
for df in df_list:
    integrate_df = pd.merge(integrate_df,df, left_index=True, right_index=True, how='left')
integrate_df = integrate_df.iloc[:,2:].dropna(how='all') # delete genes with no dN/dS scores #Feb 1 2020 bug fix
integrate_df = pd.merge(info_df,integrate_df,left_index=True,right_index=True,how='right')

Calculate the statistics of each human protein-coding gene.

In [8]:
stats_df = integrate_df.iloc[:,2:].apply(pd.DataFrame.describe, axis=1)

Save the tables. 

In [10]:
integrate_df.to_csv('../results/Ensembl98_human/human.92_species_dNdS.all_genes.tsv',sep='\t')

In [11]:
stats_df.to_csv('../results/Ensembl98_human/human.dNdS_stats.all_genes.tsv',sep='\t')

# Statistics

In [14]:
import heapq
import scipy.stats as stats

In [16]:
arr = stats_df['mean'].dropna().values

Calculate the confidence interval of the median dN/dS score

In [17]:
low = stats.binom.interval(alpha=.95,n=arr.shape[0],p=.5)[0]
high = stats.binom.interval(alpha=.95,n=arr.shape[0],p=.5)[1]
CI_low = heapq.nsmallest(low.astype(int),arr)[-1]
CI_high = heapq.nsmallest(high.astype(int),arr)[-1]

In [18]:
CI_low #lower bound of confidence interval 

0.15518286287639288

In [19]:
CI_high #higher bound of confidence interval

0.1600582979928002

In [20]:
arr.shape #number of protein-coding genes with at least one species with valid dN/dS against human

(17866,)

In [22]:
stats_df['mean'].median() # median of all human protein coding genes' average mammalian dN/dS

0.15763431379873702

# Plotting

In [23]:
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [24]:
matplotlib.rcParams['figure.dpi']= 300 #make high quality figure

In [25]:
# Creating a figure 
fig = plt.figure(figsize=(10,7.5)) # Size of a letter size paper in horizontal
fig.suptitle('Distribution of dN/dS of All Human Protein-coding Genes', fontsize=14)

# Setting subplot space
grid = plt.GridSpec(nrows=1,ncols=1)

# The subplot for distribution histogram 
distr_plot = fig.add_subplot(grid[:,:])

# Set up the bins for log scale x-axis, and get the centers
bins=np.logspace(np.log10(0.001),np.log10(10), 100)
bins_cntr = (bins[1:] + bins[:-1]) / 2

# Distribution Histograms
counts, bin_edges, ignored = distr_plot.hist(arr, bins, histtype='stepfilled', alpha=0.3, 
                                           label='dN/dS of protein-coding genes (med={0:.3f})'.format(np.median(arr)))
# Log-normal Curve for Late Development Genes
try:
    # calculate area of histograms (area under PDF should be 1)
    area_hist = ((bin_edges[1:] - bin_edges[:-1]) * counts).sum()
    shape, loc, scale = stats.lognorm.fit(arr)
    # pdf-values using cdf 
    fit_log_cntr_ = stats.lognorm.cdf(bins, shape, loc=loc, scale=scale)
    fit_log_cntr = np.diff(fit_log_cntr_)
    # plot fitted and scaled PDFs into histogram
    distr_plot.plot(bins_cntr, fit_log_cntr * counts.sum(),'b-', 
                    label='lognormal fit of dN/dS distribution', linewidth=2)
except ValueError:
    pass

# Axis labels
distr_plot.set_xlabel(xlabel='dN/dS')
distr_plot.set_ylabel(ylabel='number of genes')
distr_plot.set_xscale('log')
distr_plot.legend(loc='best')

fig.savefig('../figures/human.all_genes.pdf')
fig.savefig('../figures/human.all_genes.eps')
fig.savefig('../figures/human.all_genes.png')
plt.close()

Text(0.5, 0.98, 'Distribution of dN/dS of All Human Protein-coding Genes')

Text(0.5, 0, 'dN/dS')

Text(0, 0.5, 'number of genes')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
